In [3]:
import pandas as pd

def generate_headings_diagram(excel_file_path):
    # Load the Excel file
    excel_data = pd.ExcelFile('Geotechnical_Schema.xlsx')
    
    # Iterate over each sheet name
    for sheet_name in excel_data.sheet_names:
        # Read the sheet data into a DataFrame
        df = excel_data.parse(sheet_name)
        
        # Get the column headers
        headers = df.columns.tolist()
        
        # Generate a text diagram
        print(f"Sheet: {sheet_name}")
        print("Headings:")
        print(" + " + " + ".join(["-" * len(header) for header in headers]) + " + ")
        print(" | " + " | ".join([header.center(len(header)) for header in headers]) + " | ")
        print(" + " + " + ".join(["-" * len(header) for header in headers]) + " + ")
        print("\n")

# Example usage:
excel_file_path = 'path/to/your/excel_file.xlsx'
generate_headings_diagram(excel_file_path)

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import pandas as pd
import sqlite3
import uuid
from datetime import datetime
import os

class ExcelToSQLiteMigrator:
    def __init__(self, excel_file_path, sqlite_db_path="GeoDataBase"):
        self.excel_file_path = excel_file_path
        self.sqlite_db_path = sqlite_db_path
        self.conn = None
        self.cur = None
        
        # Storage for generated IDs to maintain relationships
        self.client_ids = {}
        self.project_ids = {}
        self.rig_ids = {}
        self.hole_ids = {}
        self.sample_ids = {}
        self.method_ids = {}
        self.geo_ids = {}
        self.cons_load_ids = {}
        self.coring_method_ids = {}
        
    def connect_db(self):
        """Connect to SQLite database"""
        self.conn = sqlite3.connect(self.sqlite_db_path)
        self.cur = self.conn.cursor()
        self.cur.execute('PRAGMA foreign_keys = ON;')
        
    def close_db(self):
        """Close database connection"""
        if self.conn:
            self.conn.commit()
            self.conn.close()
            
    def generate_id(self, prefix=""):
        """Generate unique ID"""
        return f"{prefix}_{uuid.uuid4().hex[:8]}"
    
    def safe_float(self, value):
        """Safely convert to float, return None if conversion fails"""
        if pd.isna(value) or value == '' or value is None:
            return None
        try:
            return float(value)
        except (ValueError, TypeError):
            return None
    
    def safe_int(self, value):
        """Safely convert to int, return None if conversion fails"""
        if pd.isna(value) or value == '' or value is None:
            return None
        try:
            return int(value)
        except (ValueError, TypeError):
            return None
    
    def safe_str(self, value):
        """Safely convert to string, return None if empty"""
        if pd.isna(value) or value == '' or value is None:
            return None
        return str(value).strip()
    
    def migrate_project_data(self):
        """Migrate Project sheet data"""
        try:
            df = pd.read_excel(self.excel_file_path, sheet_name='Project')
            
            for _, row in df.iterrows():
                # Generate client ID
                client_name = self.safe_str(row.get('clientName'))
                client_contact = self.safe_str(row.get('clientContact'))
                
                if client_name and client_name not in self.client_ids:
                    client_id = self.generate_id('CLIENT')
                    self.client_ids[client_name] = client_id
                    
                    # Insert client
                    self.cur.execute('''
                        INSERT INTO "_Client" ("_Client_ID", "clientName", "clientContact")
                        VALUES (?, ?, ?)
                    ''', (client_id, client_name, client_contact))
                
                # Generate project ID
                project_name = self.safe_str(row.get('projectName'))
                if project_name:
                    project_id = self.generate_id('PROJ')
                    self.project_ids[project_name] = project_id
                    
                    # Insert project
                    self.cur.execute('''
                        INSERT INTO "_Project" ("_Project_ID", "_Client_ID", "projectName", 
                                              "projectNumber", "projectCountry", "projectState", 
                                              "projectCounty", "coordinateDatum")
                        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        project_id,
                        self.client_ids.get(client_name) if client_name else None,
                        project_name,
                        self.safe_str(row.get('projectNumber')),
                        self.safe_str(row.get('projectCountry')),
                        self.safe_str(row.get('projectState')),
                        self.safe_str(row.get('projectCounty')),
                        self.safe_str(row.get('coordinateDatum'))
                    ))
                    
        except Exception as e:
            print(f"Error migrating project data: {e}")
    
    def migrate_hole_info(self):
        """Migrate HoleInfo sheet data"""
        try:
            df = pd.read_excel(self.excel_file_path, sheet_name='HoleInfo')
            
            for _, row in df.iterrows():
                hole_name = self.safe_str(row.get('holeName'))
                if not hole_name:
                    continue
                    
                # Generate rig ID if needed
                rig_desc = self.safe_str(row.get('_rigDescription'))
                if rig_desc and rig_desc not in self.rig_ids:
                    rig_id = self.generate_id('RIG')
                    self.rig_ids[rig_desc] = rig_id
                    
                    # Insert rig
                    self.cur.execute('''
                        INSERT INTO "_Rig" ("_rigID", "_rigDescription", "hammerType", 
                                          "hammerEfficiency", "miscID")
                        VALUES (?, ?, ?, ?, ?)
                    ''', (
                        rig_id,
                        rig_desc,
                        self.safe_str(row.get('hammerType')),
                        self.safe_float(row.get('hammerEfficiency')),
                        self.safe_str(row.get('miscID'))
                    ))
                
                # Generate hole ID
                hole_id = self.generate_id('HOLE')
                self.hole_ids[hole_name] = hole_id
                
                # Get first project ID (assuming single project for now)
                project_id = list(self.project_ids.values())[0] if self.project_ids else None
                
                # Insert hole info
                self.cur.execute('''
                    INSERT INTO "_HoleInfo" ("_holeID", "_rigID", "_Project_ID", "holeName", 
                                           "holeType", "topLatitude", "topLongitude", 
                                           "groundSurface", "azimuth", "angle", "bottomDepth",
                                           "timeInterval_start", "timeInterval_end", "termination")
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    hole_id,
                    self.rig_ids.get(rig_desc) if rig_desc else None,
                    project_id,
                    hole_name,
                    self.safe_str(row.get('holeType')),
                    self.safe_float(row.get('topLatitude')),
                    self.safe_float(row.get('topLongitude')),
                    self.safe_float(row.get('groundSurface')),
                    self.safe_float(row.get('azimuth')),
                    self.safe_float(row.get('angle')),
                    self.safe_float(row.get('bottomDepth')),
                    self.safe_str(row.get('timeInterval_start')),
                    self.safe_str(row.get('timeInterval_end')),
                    self.safe_str(row.get('termination'))
                ))
                
                # Handle water levels
                initial_water_depth = self.safe_float(row.get('initialWaterDepth'))
                if initial_water_depth is not None:
                    self.cur.execute('''
                        INSERT INTO "_waterLevels" ("_holeID", "waterDepth", "TimeInterval_start", "TimeInterval_end")
                        VALUES (?, ?, ?, ?)
                    ''', (
                        hole_id,
                        initial_water_depth,
                        self.safe_str(row.get('timeInterval_start')),
                        self.safe_str(row.get('timeInterval_end'))
                    ))
                
                # Handle cave in
                cave_in_depth = self.safe_float(row.get('caveInDepth'))
                if cave_in_depth is not None:
                    self.cur.execute('''
                        INSERT INTO "_caveIn" ("_holeID", "caveInDepth", "TimeInterval_start", "TimeInterval_end")
                        VALUES (?, ?, ?, ?)
                    ''', (
                        hole_id,
                        cave_in_depth,
                        self.safe_str(row.get('timeInterval_start')),
                        self.safe_str(row.get('timeInterval_end'))
                    ))
                
                # Handle drill method
                drill_method = self.safe_str(row.get('drillMethod'))
                if drill_method:
                    self.cur.execute('''
                        INSERT INTO "DrillMethod" ("_holeID", "drillMethod", "rodType", "additives", "misc")
                        VALUES (?, ?, ?, ?, ?)
                    ''', (
                        hole_id,
                        drill_method,
                        self.safe_str(row.get('rodType')),
                        self.safe_str(row.get('Additives')),
                        self.safe_str(row.get('misc'))
                    ))
                    
        except Exception as e:
            print(f"Error migrating hole info: {e}")
    
    def migrate_test_methods(self):
        """Migrate TestMethod sheet data"""
        try:
            df = pd.read_excel(self.excel_file_path, sheet_name='TestMethod')
            
            for _, row in df.iterrows():
                method_name = self.safe_str(row.get('methodName'))
                if not method_name:
                    continue
                    
                method_id = self.generate_id('METHOD')
                self.method_ids[method_name] = method_id
                
                self.cur.execute('''
                    INSERT INTO "TestMethod" ("_Method_ID", "methodName", "governingBody", "units", "modification")
                    VALUES (?, ?, ?, ?, ?)
                ''', (
                    method_id,
                    method_name,
                    self.safe_str(row.get('description')),
                    self.safe_str(row.get('governingBody')),
                    self.safe_str(row.get('units')),
                    self.safe_str(row.get('modification'))
                ))
                
        except Exception as e:
            print(f"Error migrating test methods: {e}")
    
    def migrate_samples(self):
        """Migrate Samples sheet data"""
        try:
            df = pd.read_excel(self.excel_file_path, sheet_name='Samples')
            
            for _, row in df.iterrows():
                hole_name = self.safe_str(row.get('Hole Name'))
                sample_name = self.safe_str(row.get('Sample Name'))
                
                if not hole_name or not sample_name:
                    continue
                
                sample_id = self.generate_id('SAMPLE')
                sample_key = f"{hole_name}_{sample_name}"
                self.sample_ids[sample_key] = sample_id
                
                hole_id = self.hole_ids.get(hole_name)
                if not hole_id:
                    continue
                
                # Insert sample
                self.cur.execute('''
                    INSERT INTO "_Samples" ("_Sample_ID", "_holeID", "sampleName", 
                                          "pos_topDepth", "pos_bottomDepth", "sampleMethod")
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (
                    sample_id,
                    hole_id,
                    sample_name,
                    self.safe_float(row.get('topDepth')),
                    self.safe_float(row.get('bottomDepth')),
                    self.safe_str(row.get('sampleMethod'))
                ))
                
                # Handle individual test results
                self._insert_sample_test_data(sample_id, row)
                
        except Exception as e:
            print(f"Error migrating samples: {e}")
    
    def _insert_sample_test_data(self, sample_id, row):
        """Insert individual test data for a sample"""
        # Get a default method ID (you might want to create specific methods)
        default_method_id = list(self.method_ids.values())[0] if self.method_ids else self.generate_id('METHOD')
        
        # SPT Data
        spt_1 = self.safe_int(row.get('SPT_1'))
        if spt_1 is not None:
            self.cur.execute('''
                INSERT INTO "_SPT" ("_Sample_ID", "_Method_ID", "blowCount_index1", 
                                  "blowCount_index2", "blowCount_index3", "blowCount_index4", "recovery")
                VALUES (?, ?, ?, ?, ?, ?, ?)
            ''', (
                sample_id,
                default_method_id,
                spt_1,
                self.safe_int(row.get('SPT_2')),
                self.safe_int(row.get('SPT_3')),
                self.safe_int(row.get('SPT_4')),
                self.safe_float(row.get('recovery'))
            ))
        
        # Moisture Content
        moisture_content = self.safe_float(row.get('moistureContent'))
        if moisture_content is not None:
            self.cur.execute('''
                INSERT INTO "MoistureContent" ("_Sample_ID", "_Method_ID", "moistureContent")
                VALUES (?, ?, ?)
            ''', (sample_id, default_method_id, moisture_content))
        
        # Atterberg Limits
        pl = self.safe_float(row.get('PL'))
        ll = self.safe_float(row.get('LL'))
        pi = self.safe_float(row.get('PI'))
        if any([pl, ll, pi]):
            self.cur.execute('''
                INSERT INTO "AtterbergLimits" ("_Sample_ID", "_Method_ID", "plasticLimit", "liquidLimit", "plasticityIndex")
                VALUES (?, ?, ?, ?, ?)
            ''', (sample_id, default_method_id, pl, ll, pi))
        
        # 200 Wash
        passing_200 = self.safe_float(row.get('passing200'))
        if passing_200 is not None:
            self.cur.execute('''
                INSERT INTO "200wash" ("_Sample_ID", "_Method_ID", "passing200")
                VALUES (?, ?, ?)
            ''', (sample_id, default_method_id, passing_200))
        
        # Pocket Penetrometer
        pocket_pen = self.safe_float(row.get('pocketPenReading'))
        if pocket_pen is not None:
            self.cur.execute('''
                INSERT INTO "Pocket_Pen" ("_Sample_ID", "_Method_ID", "reading")
                VALUES (?, ?, ?)
            ''', (sample_id, default_method_id, pocket_pen))
        
        # Torvane
        torvane_reading = self.safe_float(row.get('torvaneReading'))
        if torvane_reading is not None:
            self.cur.execute('''
                INSERT INTO "torvane" ("_Sample_ID", "_Method_ID", "reading")
                VALUES (?, ?, ?)
            ''', (sample_id, default_method_id, torvane_reading))
        
        # Field Soil Description
        primary_comp = self.safe_str(row.get('primaryComp'))
        if primary_comp:
            geo_id = self.generate_id('GEO')
            self.cur.execute('''
                INSERT INTO "fieldSoilDesc" ("_Sample_ID", "_Geo_ID", "color", "primaryComp", 
                                           "secondaryComp", "secondaryCompMod", "organicContent", "visualMoisture")
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                sample_id,
                geo_id,
                self.safe_str(row.get('color')),
                primary_comp,
                self.safe_str(row.get('secondaryComp')),
                self.safe_str(row.get('secondaryCompMod')),
                self.safe_float(row.get('organicContent')),
                self.safe_str(row.get('visualMoisture'))
            ))
    
    def migrate_gradation(self):
        """Migrate Gradation sheet data"""
        try:
            df = pd.read_excel(self.excel_file_path, sheet_name='Gradation')
            
            default_method_id = list(self.method_ids.values())[0] if self.method_ids else self.generate_id('METHOD')
            
            for _, row in df.iterrows():
                boring = self.safe_str(row.get('Boring'))
                sample = self.safe_str(row.get('Sample'))
                
                if not boring or not sample:
                    continue
                
                sample_key = f"{boring}_{sample}"
                sample_id = self.sample_ids.get(sample_key)
                
                if not sample_id:
                    continue
                
                self.cur.execute('''
                    INSERT INTO "Gradation" ("_Sample_ID", "_Method_ID", "retNo4", "retNo10", 
                                           "retNo20", "retNo40", "retNo60", "retNo100", "retNo140", "retNo200")
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    sample_id,
                    default_method_id,
                    self.safe_float(row.get('retNo4')),
                    self.safe_float(row.get('retNo10')),
                    self.safe_float(row.get('retNo20')),
                    self.safe_float(row.get('retNo40')),
                    self.safe_float(row.get('retNo60')),
                    self.safe_float(row.get('retNo100')),
                    self.safe_float(row.get('retNo140')),
                    self.safe_float(row.get('retNo200'))
                ))
                
        except Exception as e:
            print(f"Error migrating gradation: {e}")
    
    def migrate_consolidation(self):
        """Migrate Consolidation sheet data"""
        try:
            df = pd.read_excel(self.excel_file_path, sheet_name='Consolidation')
            
            default_method_id = list(self.method_ids.values())[0] if self.method_ids else self.generate_id('METHOD')
            
            for _, row in df.iterrows():
                boring = self.safe_str(row.get('Boring'))
                sample = self.safe_str(row.get('Sample'))
                
                if not boring or not sample:
                    continue
                
                sample_key = f"{boring}_{sample}"
                sample_id = self.sample_ids.get(sample_key)
                
                if not sample_id:
                    continue
                
                cons_load_id = self.generate_id('CONSLOAD')
                
                self.cur.execute('''
                    INSERT INTO "Consolidation" ("_Sample_ID", "_Method_ID", "_Cons_Load_ID", 
                                               "initialVoidRatio", "compressionIndex", "recompressionIndex",
                                               "overburdenPressure", "preconsolidationPressure")
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    sample_id,
                    default_method_id,
                    cons_load_id,
                    self.safe_float(row.get('initialVoidRatio')),
                    self.safe_float(row.get('compressionIndex')),
                    self.safe_float(row.get('recompressionIndex')),
                    self.safe_float(row.get('overburdenPressure')),
                    self.safe_float(row.get('preconsolidationPressure'))
                ))
                
        except Exception as e:
            print(f"Error migrating consolidation: {e}")
    
    def migrate_triaxial_tests(self):
        """Migrate triaxial test data (UU, CU, DS)"""
        test_types = ['uuTest', 'cuTest', 'dsTest']
        
        for test_type in test_types:
            try:
                df = pd.read_excel(self.excel_file_path, sheet_name=test_type)
                
                default_method_id = list(self.method_ids.values())[0] if self.method_ids else self.generate_id('METHOD')
                
                for _, row in df.iterrows():
                    sample_id = self.safe_str(row.get('_Sample_ID'))
                    if not sample_id:
                        continue
                    
                    if test_type == 'uuTest':
                        self.cur.execute('''
                            INSERT INTO "uuTest" ("_Sample_ID", "_Method_ID", "uuSample", "intWC", "intDryDen",
                                                "intSat", "intVoid", "testWC", "testDryDen", "testSat", "testVoid",
                                                "strainRate", "backPres", "cellPres", "failStress", "ultStress",
                                                "sigma1", "sigma3", "totPhi", "totC", "effPhi", "effC")
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                        ''', (
                            sample_id, default_method_id, self.safe_float(row.get('uuSample')),
                            self.safe_float(row.get('intWC')), self.safe_float(row.get('intDryDen')),
                            self.safe_float(row.get('intSat')), self.safe_float(row.get('intVoid')),
                            self.safe_float(row.get('testWC')), self.safe_float(row.get('testDryDen')),
                            self.safe_float(row.get('testSat')), self.safe_float(row.get('testVoid')),
                            self.safe_float(row.get('strainRate')), self.safe_float(row.get('backPres')),
                            self.safe_float(row.get('cellPres')), self.safe_float(row.get('failStress')),
                            self.safe_float(row.get('ultStress')), self.safe_float(row.get('sigma1')),
                            self.safe_float(row.get('sigma3')), self.safe_float(row.get('totPhi')),
                            self.safe_float(row.get('totC')), self.safe_float(row.get('effPhi')),
                            self.safe_float(row.get('effC'))
                        ))
                    
                    # Similar patterns for cuTest and dsTest...
                    
            except Exception as e:
                print(f"Error migrating {test_type}: {e}")
    
    def migrate_well_data(self):
        """Migrate well construction and readings data"""
        try:
            # Well Construction
            df = pd.read_excel(self.excel_file_path, sheet_name='WellConstr')
            
            for _, row in df.iterrows():
                boring = self.safe_str(row.get('Boring'))
                hole_id = self.hole_ids.get(boring)
                
                if not hole_id:
                    continue
                
                self.cur.execute('''
                    INSERT INTO "WellConstr" ("_holeID", "material", "pos_topDepth", "pos_bottomDepth")
                    VALUES (?, ?, ?, ?)
                ''', (
                    hole_id,
                    self.safe_str(row.get('material')),
                    self.safe_float(row.get('pos_topDepth')),
                    self.safe_float(row.get('pos_bottomDepth'))
                ))
                
            # Well Readings
            df = pd.read_excel(self.excel_file_path, sheet_name='WellReadings')
            
            for _, row in df.iterrows():
                boring = self.safe_str(row.get('Boring'))
                hole_id = self.hole_ids.get(boring)
                
                if not hole_id:
                    continue
                
                self.cur.execute('''
                    INSERT INTO "WellReadings" ("_holeID", "reading", "temp", "TimeInterval")
                    VALUES (?, ?, ?, ?)
                ''', (
                    hole_id,
                    self.safe_float(row.get('reading')),
                    self.safe_float(row.get('temp')),
                    self.safe_str(row.get('TimeInterval'))
                ))
                
        except Exception as e:
            print(f"Error migrating well data: {e}")
    
    def run_migration(self):
        """Run the complete migration process"""
        print("Starting Excel to SQLite migration...")
        
        try:
            self.connect_db()
            
            # Migration order is important due to foreign key constraints
            print("Migrating project data...")
            self.migrate_project_data()
            
            print("Migrating hole information...")
            self.migrate_hole_info()
            
            print("Migrating test methods...")
            self.migrate_test_methods()
            
            print("Migrating samples...")
            self.migrate_samples()
            
            print("Migrating gradation data...")
            self.migrate_gradation()
            
            print("Migrating consolidation data...")
            self.migrate_consolidation()
            
            print("Migrating triaxial tests...")
            self.migrate_triaxial_tests()
            
            print("Migrating well data...")
            self.migrate_well_data()
            
            print("Migration completed successfully!")
            
        except Exception as e:
            print(f"Migration failed: {e}")
            if self.conn:
                self.conn.rollback()
        finally:
            self.close_db()


# Usage example
if __name__ == "__main__":
    # Replace with your actual Excel file path
    excel_file = "Geotechnical_Schema.xlsx"
    
    # Check if file exists
    if not os.path.exists(excel_file):
        print(f"Excel file not found: {excel_file}")
        print("Please provide the correct path to your Excel file.")
    else:
        # Create migrator instance
        migrator = ExcelToSQLiteMigrator(excel_file)
        
        # Run migration
        migrator.run_migration()

Starting Excel to SQLite migration...
Migrating project data...
Error migrating project data: no such table: _Client
Migrating hole information...
Error migrating hole info: no such table: _HoleInfo
Migrating test methods...
Error migrating test methods: no such table: TestMethod
Migrating samples...
Migrating gradation data...
Migrating consolidation data...
Migrating triaxial tests...
Migrating well data...
Migration completed successfully!
